<a href="https://colab.research.google.com/github/ColeJMitchell/machine-learning-stock-prediction/blob/main/model_notebooks/model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [450]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, time
from termcolor import colored
from IPython.display import clear_output
!pip install tensorflow
!git clone https://github.com/ColeJMitchell/machine-learning-stock-prediction
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import random
clear_output()

# **Loading The Data**

In [451]:
test_stocks = ['AAPL', 'NVDA', 'MSTR', 'MP', 'PG', 'GLD', 'TSLA', 'AMC', 'META', 'HIMS']
closing_prices = {}
start_date = end_date - datetime.timedelta(days=210)
end_date = datetime.datetime.today()
for ticker in test_stocks:
    data = yf.download(ticker, start=start_date, end=end_date)
    closing_prices[ticker] = data["Close"].values.squeeze()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# **Paper Trading Method 1 : Using The LSTM Model**

In [452]:
#Load the LSTM model and simulate it by paper trading
model = load_model("machine-learning-stock-prediction/saved_models/stock_prediction.h5", compile = False)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,513 (119.19 KB)

 Trainable params: 30,321 (118.44 KB)

 Non-trainable params: 192 (768.00 B)

In [453]:
# Number of closing price values for each stock ticker
print(f"Number of Closing Price Data Points: {len(closing_prices['AAPL'])}")

Number of Closing Price Data Points: 144


## Preprocessing The Data With Sliding Windows To Match Shape Required By The LSTM

In [454]:
window_size = 30
closing_price_windows = {}
# Stocks need to remain separate for the paper trading
for ticker in test_stocks:
    closing_price_windows[ticker] = []
    for i in range(0, 144 - window_size + 1, window_size):
        window = closing_prices[ticker][i:i + window_size]
        closing_price_windows[ticker].append(np.array(window))

print(f"Shape of Closing Price Sliding Windows: {np.array(closing_price_windows[test_stocks[0]]).shape}")

Shape of Closing Price Sliding Windows: (4, 30)


## Trading Algorithm Using The LSTM's Predictions

In [455]:
# A list of stocks purchased by the LSTM model is saved so it can know when to sell,
# the model starts with the same amount of money as the random model - 10,000 dollars
portfolio = {}
initial_wallet = 10000

for stock in test_stocks:
    portfolio[stock] = {
        "Remaining_Cash": 1000,
        "Bought": 0,
        "Purchase_Prices": [],
        "Final_Amount_Sold": 0
    }

# If a stock is predicted to rise in the future (larger value than final entry in sliding window),
# it is bought by the model, $100 of the allocated money for the stock is bought.
# If the stock is predicted to fall below the purchase price the stock is sold at the baseline price (last day of the sliding window)
for stock in test_stocks:
    windows = closing_price_windows[stock]
    for window in windows:
        baseline_price = window[-1]
        scaler = MinMaxScaler(feature_range=(0, 1))
        normalized_window = scaler.fit_transform(window.reshape(-1, 1))

        # The model makes a prediction based on the window and compares it to the last day in the window to decide whether to buy, sell, or do nothing
        prediction = model.predict(normalized_window.reshape(1, 30), verbose=0)
        prediction = scaler.inverse_transform(prediction).item()

        current_remaining_cash = portfolio[stock]["Remaining_Cash"]
        current_bought = portfolio[stock]["Bought"]
        purchase_prices = portfolio[stock]["Purchase_Prices"]

        # When the model thinks the price will go up
        if baseline_price < prediction:
            if current_remaining_cash >= 200:
                # Stocks are bought in 200 dollar increments
                portfolio[stock]["Remaining_Cash"] -= 200
                portfolio[stock]["Bought"] += 200
                purchase_prices.append(baseline_price)

        # When the model things the price will go down
        elif baseline_price > prediction:
            if current_bought > 0:
                # Copy the list to avoid mutation during iteration
                for price in purchase_prices[:]:
                    # Sells the bought stock at the baseline price if the prediction anticipates that the stock price will go down.
                    # The final bought calculation only considers the final liquidation so 100 is removed when sold.
                    if price > prediction:
                        percentage_change = baseline_price / price
                        portfolio[stock]["Remaining_Cash"] += 200 * percentage_change
                        portfolio[stock]["Bought"] -= 200
                        purchase_prices.remove(price)
                        break
        else:
            pass

    # Sells all the stocks that were bought at the very end
    final_stock_price = float(windows[-1][-1])
    for purchase_price in portfolio[stock]["Purchase_Prices"]:
        percentage_change = final_stock_price / purchase_price
        sale_proceeds = 200 * percentage_change
        portfolio[stock]["Final_Amount_Sold"] += sale_proceeds


## Profit And Loss Margins For The LSTM Model

In [456]:
# Output the final earnings and losses
for stock in test_stocks:
  print(f"{stock} detailed stats:")
  print(f"Final Remaining Cash: {int(portfolio[stock]['Remaining_Cash'])}$")
  print(f"Stock Quantity Bought: {portfolio[stock]['Bought']}$")
  print(f"Stock Purchase Prices: {[int(x) for x in portfolio[stock]['Purchase_Prices']]}")
  print(f"Final Amount Sold: {int(portfolio[stock]['Final_Amount_Sold'])}$")
  print("\n")

AAPL detailed stats:
Final Remaining Cash: 969$
Stock Quantity Bought: 0$
Stock Purchase Prices: []
Final Amount Sold: 0$


NVDA detailed stats:
Final Remaining Cash: 997$
Stock Quantity Bought: 0$
Stock Purchase Prices: []
Final Amount Sold: 0$


MSTR detailed stats:
Final Remaining Cash: 800$
Stock Quantity Bought: 200$
Stock Purchase Prices: [263]
Final Amount Sold: 225$


MP detailed stats:
Final Remaining Cash: 800$
Stock Quantity Bought: 200$
Stock Purchase Prices: [23]
Final Amount Sold: 200$


PG detailed stats:
Final Remaining Cash: 600$
Stock Quantity Bought: 400$
Stock Purchase Prices: [156, 161]
Final Amount Sold: 405$


GLD detailed stats:
Final Remaining Cash: 1000$
Stock Quantity Bought: 0$
Stock Purchase Prices: []
Final Amount Sold: 0$


TSLA detailed stats:
Final Remaining Cash: 987$
Stock Quantity Bought: 0$
Stock Purchase Prices: []
Final Amount Sold: 0$


AMC detailed stats:
Final Remaining Cash: 985$
Stock Quantity Bought: 0$
Stock Purchase Prices: []
Final Amount

In [457]:
# Final amount made and lost
total = 0
for stock in test_stocks:
  print(f"{stock} overall stats:")
  print(f"Starting Amount: 1000$")
  total += float(portfolio[stock]['Final_Amount_Sold']) + portfolio[stock]['Remaining_Cash']
  print(f"Ending Amount: {int(portfolio[stock]['Final_Amount_Sold']) + int(portfolio[stock]['Remaining_Cash'])}$")
  print("\n")
print("-----------------------------------------------------------------------------------------------")
percent_change = total / 10000
print("Total Beginning: 10,000$\n")
print(f"Total End: {int(total)}$\n")
print(f"Percent Change From Initial Wallet: {round(percent_change,3)*100}%")

AAPL overall stats:
Starting Amount: 1000$
Ending Amount: 969$


NVDA overall stats:
Starting Amount: 1000$
Ending Amount: 997$


MSTR overall stats:
Starting Amount: 1000$
Ending Amount: 1025$


MP overall stats:
Starting Amount: 1000$
Ending Amount: 1000$


PG overall stats:
Starting Amount: 1000$
Ending Amount: 1005$


GLD overall stats:
Starting Amount: 1000$
Ending Amount: 1000$


TSLA overall stats:
Starting Amount: 1000$
Ending Amount: 987$


AMC overall stats:
Starting Amount: 1000$
Ending Amount: 985$


META overall stats:
Starting Amount: 1000$
Ending Amount: 1000$


HIMS overall stats:
Starting Amount: 1000$
Ending Amount: 1027$


-----------------------------------------------------------------------------------------------
Total Beginning: 10,000$

Total End: 9998$

Percent Change From Initial Wallet: 100.0%


# **Paper Trading Method 2 : Random Behavior**

In [458]:
# A list of the randomly purchased stocks is kept for when the model sells the stocks at the end
random_portfolio = {}
for stock in test_stocks:
    random_portfolio[stock] = {
        "Remaining_Cash": 1000,
        "Bought": 0,
        "Purchase_Prices": [],
        "Final_Amount_Sold": 0
    }

# The window structure from the evaluation of the LSTM model is maintained so that the random model performs actions at the same time intervals
for stock in test_stocks:
    windows = closing_price_windows[stock]
    for window in windows:
        baseline_price = window[-1]
        current_remaining_cash = random_portfolio[stock]["Remaining_Cash"]
        current_bought = random_portfolio[stock]["Bought"]
        purchase_prices = random_portfolio[stock]["Purchase_Prices"]

        random_value = random.choice([0, 1, 2])

        if random_value == 0:
            if current_remaining_cash >= 200:
                # Stocks are still bought in 200 dollar increments
                random_portfolio[stock]["Remaining_Cash"] -= 200
                random_portfolio[stock]["Bought"] += 200
                purchase_prices.append(baseline_price)

        elif random_value == 1:
            if current_bought > 0 and purchase_prices:
                # The selling logic is also the same as the LSTM except it picks a stock at random out of the purchased list
                random_index = random.randint(0, len(purchase_prices) - 1)
                purchase_price = purchase_prices.pop(random_index)
                percentage_change = baseline_price / purchase_price
                random_portfolio[stock]["Remaining_Cash"] += 200 * percentage_change
                random_portfolio[stock]["Bought"] -= 200

        else:
            pass

    # Liquidates all the stocks that were bought at the very end
    final_stock_price = float(windows[-1][-1])
    for purchase_price in random_portfolio[stock]["Purchase_Prices"]:
        percentage_change = final_stock_price / purchase_price
        sale_proceeds = 100 * percentage_change
        random_portfolio[stock]["Final_Amount_Sold"] += sale_proceeds


## Profit And Loss Margins For The Random Algorithm

In [459]:
# Output the final earnings and losses
for stock in test_stocks:
  print(f"{stock} detailed stats:")
  print(f"Final Remaining Cash: {int(random_portfolio[stock]['Remaining_Cash'])}$")
  print(f"Stock Quantity Bought: {random_portfolio[stock]['Bought']}$")
  print(f"Stock Purchase Prices: {[int(x) for x in random_portfolio[stock]['Purchase_Prices']]}")
  print(f"Final Amount Sold: {int(random_portfolio[stock]['Final_Amount_Sold'])}$")
  print("\n")

AAPL detailed stats:
Final Remaining Cash: 600$
Stock Quantity Bought: 400$
Stock Purchase Prices: [234, 233]
Final Amount Sold: 169$


NVDA detailed stats:
Final Remaining Cash: 800$
Stock Quantity Bought: 200$
Stock Purchase Prices: [114]
Final Amount Sold: 100$


MSTR detailed stats:
Final Remaining Cash: 800$
Stock Quantity Bought: 200$
Stock Purchase Prices: [296]
Final Amount Sold: 100$


MP detailed stats:
Final Remaining Cash: 800$
Stock Quantity Bought: 200$
Stock Purchase Prices: [23]
Final Amount Sold: 100$


PG detailed stats:
Final Remaining Cash: 600$
Stock Quantity Bought: 400$
Stock Purchase Prices: [177, 170]
Final Amount Sold: 185$


GLD detailed stats:
Final Remaining Cash: 812$
Stock Quantity Bought: 200$
Stock Purchase Prices: [242]
Final Amount Sold: 117$


TSLA detailed stats:
Final Remaining Cash: 800$
Stock Quantity Bought: 200$
Stock Purchase Prices: [403]
Final Amount Sold: 67$


AMC detailed stats:
Final Remaining Cash: 1000$
Stock Quantity Bought: 0$
Stock 

In [460]:
# Final amount made and lost
total = 0
for stock in test_stocks:
  print(f"{stock} overall stats:")
  print(f"Starting Amount: 1000")
  total += int(random_portfolio[stock]['Final_Amount_Sold']) + random_portfolio[stock]['Remaining_Cash']
  print(f"Ending Amount: {int(random_portfolio[stock]['Final_Amount_Sold']) + random_portfolio[stock]['Remaining_Cash']}$")
  print("\n")
print("-----------------------------------------------------------------------------------------------")
percent_change = total / 10000
print("Total Beginning: 10,000$\n")
print(f"Total End: {total}$\n")
print(f"Percent Change From Initial Wallet: {round(percent_change,3)*100}%")

AAPL overall stats:
Starting Amount: 1000
Ending Amount: 769$


NVDA overall stats:
Starting Amount: 1000
Ending Amount: 900$


MSTR overall stats:
Starting Amount: 1000
Ending Amount: 900$


MP overall stats:
Starting Amount: 1000
Ending Amount: 900$


PG overall stats:
Starting Amount: 1000
Ending Amount: 785$


GLD overall stats:
Starting Amount: 1000
Ending Amount: 929.1547828700907$


TSLA overall stats:
Starting Amount: 1000
Ending Amount: 867$


AMC overall stats:
Starting Amount: 1000
Ending Amount: 1000$


META overall stats:
Starting Amount: 1000
Ending Amount: 870.0511004172073$


HIMS overall stats:
Starting Amount: 1000
Ending Amount: 1000$


-----------------------------------------------------------------------------------------------
Total Beginning: 10,000$

Total End: 8920.205883287297$

Percent Change From Initial Wallet: 89.2%
